In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                /failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.7.9|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|3_73_pypy|5_73_pypy|4_73_pypy|2_73_pypy|1_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']
  

In [2]:
!pip install folium
import folium

     |████████████████████████████████| 94 kB 5.1 MB/s  eta 0:00:01


## 1. Download and Explore Dataset


In [3]:
table_LA = pd.read_html('http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_LA_Neighborhoods_Data')

In [4]:
LA = table_LA[2]
LA.head()

,LA_Nbhd,Income,Schools,Diversity,Age,Homes,Vets,Asian,Black,Latino,White,Population,Area,Longitude,Latitude
0,Adams_Normandie,29606,691,0.6,26,0.26,0.05,0.05,0.25,0.62,0.06,31068,0.8,-118.300270,34.030970
1,Arleta,65649,719,0.4,29,0.29,0.07,0.11,0.02,0.72,0.13,31068,3.1,-118.430015,34.240603
2,Arlington_Heights,31423,687,0.8,31,0.31,0.05,0.13,0.25,0.57,0.05,22106,1.0,-118.320109,34.043611
3,Atwater_Village,53872,762,0.9,34,0.34,0.06,0.20,0.01,0.51,0.22,14888,1.8,-118.265808,34.124908
4,Baldwin_Hills/Crenshaw,37948,656,0.4,36,0.36,0.10,0.05,0.71,0.17,0.03,30123,3.0,-118.366700,34.019090


In [5]:
LA = LA.rename(columns={"LA_Nbhd": "Neighborhood"})

# Load and explore the data

Next, let's load the data.


# Use geopy library to get the latitude and longitude values of LA.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [8]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles, CA are 34.0536909, -118.242766.


# Create a map of New York with neighborhoods superimposed on top.


In [10]:
# create map of New York using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(LA['Latitude'], LA['Longitude'], LA['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [11]:
#### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'MIGCUI15BKFFSTLB4KKEEPJOXT3IJL3MRWNX5KN14EJ1N1FL' # your Foursquare ID
CLIENT_SECRET = '3VEHOJXAMMQ3Q1HR5JAQ5PS3LYLICQKXLALVMSCHCD41DCNW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MIGCUI15BKFFSTLB4KKEEPJOXT3IJL3MRWNX5KN14EJ1N1FL
CLIENT_SECRET:3VEHOJXAMMQ3Q1HR5JAQ5PS3LYLICQKXLALVMSCHCD41DCNW


# Let's create a function to repeat the same process to all the neighborhoods in LA


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Now write the code to run the above function on each neighborhood


In [18]:
# type your answer here
la_venues = getNearbyVenues(names=LA['Neighborhood'],
                            latitudes=LA['Latitude'],
                            longitudes=LA['Longitude']
                            )

Adams_Normandie
Arleta
Arlington_Heights
Atwater_Village
Baldwin_Hills/Crenshaw
Bel-Air
Beverly_Crest
Beverly_Grove
Beverlywood
Boyle_Heights
Brentwood
Broadway_Manchester
Canoga_Park
Carthay
Central_Alameda
Century_City
Chatsworth
Chesterfield_Square
Cheviot_Hills
Chinatown
Cypress_Park
Del_Rey
Downtown
Eagle_Rock
East_Hollywood
Echo_Park
El_Sereno
Elysian_Park
Elysian_Valley
Encino
Exposition_Park
Fairfax
Florence
Glassell_Park
Gramercy_Park
Granada_Hills
Green_Meadows
Hancock_Park
Harbor_City
Harbor_Gateway
Harvard_Heights
Harvard_Park
Highland_Park
Historic_South_Central
Hollywood
Hollywood_Hills
Hollywood_Hills_West
Hyde_Park
Jefferson_Park
Koreatown
Lake_Balboa
Lake_View_Terrace
Larchmont
Leimert_Park
Lincoln_Heights
Los_Feliz
Manchester_Square
Mar_Vista
Mid_City
Mid_Wilshire
Mission_Hills
Montecito_Heights
Mount_Washington
North_Hills
North_Hollywood
Northridge
Pacific_Palisades
Pacoima
Palms
Panorama_City
Pico_Robertson
Pico_Union
Playa_del_Rey
Playa_Vista
Porter_Ranch
Rancho_P

In [20]:
print(la_venues.shape)
la_venues.head()

(1869, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams_Normandie,34.03097,-118.30027,Orange Door Sushi,34.032485,-118.299368,Sushi Restaurant
1,Adams_Normandie,34.03097,-118.30027,Shell,34.033095,-118.300025,Gas Station
2,Adams_Normandie,34.03097,-118.30027,Little Xian,34.032292,-118.299465,Sushi Restaurant
3,Adams_Normandie,34.03097,-118.30027,Sushi Delight,34.032501,-118.299454,Sushi Restaurant
4,Adams_Normandie,34.03097,-118.30027,Tacos La Estrella,34.032230,-118.300757,Taco Place


Let's check how many venues were returned for each neighborhood

In [21]:
la_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adams_Normandie,12,12,12,12,12,12
Arleta,4,4,4,4,4,4
Arlington_Heights,6,6,6,6,6,6
Atwater_Village,39,39,39,39,39,39
Baldwin_Hills/Crenshaw,2,2,2,2,2,2
Bel-Air,1,1,1,1,1,1
Beverly_Crest,1,1,1,1,1,1
Beverly_Grove,100,100,100,100,100,100
Beverlywood,5,5,5,5,5,5


# Let's find out how many unique categories can be curated from all the returned venues

In [23]:
print('There are {} uniques categories.'.format(len(la_venues['Venue Category'].unique())))

There are 266 uniques categories.


# 3. Analyze Each Neighborhood

In [25]:
# one hot encoding
la_onehot = pd.get_dummies(la_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['Neighborhood'] = la_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Classroom,College Quad,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Romanian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,School,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Taiwanese Restaurant,Tanning Salon,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trail,Train Station,Transportation Service,Tunnel,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [27]:
la_grouped = la_onehot.groupby('Neighborhood').mean().reset_index()
la_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Classroom,College Quad,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Romanian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,School,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Taiwanese Restaurant,Tanning Salon,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trail,Train Station,Transportation Service,Tunnel,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adams_Normandie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [28]:
la_grouped.shape

(108, 266)

# Let's print each neighborhood along with the top 5 most common venues


In [30]:
num_top_venues = 5

for hood in la_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = la_grouped[la_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adams_Normandie----
              venue  freq
0  Sushi Restaurant  0.25
1       Gas Station  0.08
2        Playground  0.08
3          Pharmacy  0.08
4     Grocery Store  0.08


----Arleta----
               venue  freq
0      Historic Site  0.25
1  Convenience Store  0.25
2             Bakery  0.25
3        Video Store  0.25
4          Pet Store  0.00


----Arlington_Heights----
                 venue  freq
0       Shop & Service  0.17
1           Donut Shop  0.17
2                 Café  0.17
3        Grocery Store  0.17
4  Rental Car Location  0.17


----Atwater_Village----
                  venue  freq
0     Mobile Phone Shop  0.08
1  Fast Food Restaurant  0.05
2            Food Truck  0.05
3    Chinese Restaurant  0.05
4         Grocery Store  0.05


----Baldwin_Hills/Crenshaw----
            venue  freq
0     Flower Shop   0.5
1  Clothing Store   0.5
2     Yoga Studio   0.0
3        Pharmacy   0.0
4    Noodle House   0.0


----Bel-Air----
                   venue  freq
0      

First, let's write a function to sort the venues in descending order.


In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = la_grouped['Neighborhood']

for ind in np.arange(la_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams_Normandie,Sushi Restaurant,Grocery Store,Home Service,Pharmacy,Latin American Restaurant,Gas Station,Taco Place,Playground,Food,Park
1,Arleta,Bakery,Video Store,Historic Site,Convenience Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop
2,Arlington_Heights,Rental Car Location,Café,Shop & Service,Donut Shop,Grocery Store,Restaurant,Food Court,Food & Drink Shop,Food,Falafel Restaurant
3,Atwater_Village,Mobile Phone Shop,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Food Truck,Breakfast Spot,Sushi Restaurant,Gastropub,Steakhouse,Miscellaneous Shop
4,Baldwin_Hills/Crenshaw,Flower Shop,Clothing Store,Women's Store,Farmers Market,French Restaurant,Fountain,Food Truck,Food Stand,Food Service,Food Court
5,Bel-Air,Dive Bar,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Women's Store,Falafel Restaurant
6,Beverly_Crest,Food Truck,Women's Store,Falafel Restaurant,French Restaurant,Fountain,Food Stand,Food Service,Food Court,Food & Drink Shop,Food
7,Beverly_Grove,Clothing Store,Juice Bar,Cosmetics Shop,Gym / Fitness Center,Vegetarian / Vegan Restaurant,Lingerie Store,Café,Seafood Restaurant,Sushi Restaurant,Chinese Restaurant
8,Beverlywood,Coffee Shop,Food Truck,Park,Restaurant,Boutique,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market
9,Boyle_Heights,Grocery Store,Juice Bar,Gym,Liquor Store,Taco Place,Theater,Breakfast Spot,Mobile Phone Shop,Seafood Restaurant,Home Service


# 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [35]:
# set number of clusters
kclusters = 5

la_grouped_clustering = la_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [36]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

la_merged = LA

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
la_merged = la_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

la_merged.head() # check the last columns!

,Neighborhood,Income,Schools,Diversity,Age,Homes,Vets,Asian,Black,Latino,White,Population,Area,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams_Normandie,29606,691,0.6,26,0.26,0.05,0.05,0.25,0.62,0.06,31068,0.8,-118.300270,34.030970,0.0,Sushi Restaurant,Grocery Store,Home Service,Pharmacy,Latin American Restaurant,Gas Station,Taco Place,Playground,Food,Park
1,Arleta,65649,719,0.4,29,0.29,0.07,0.11,0.02,0.72,0.13,31068,3.1,-118.430015,34.240603,0.0,Bakery,Video Store,Historic Site,Convenience Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop
2,Arlington_Heights,31423,687,0.8,31,0.31,0.05,0.13,0.25,0.57,0.05,22106,1.0,-118.320109,34.043611,0.0,Rental Car Location,Café,Shop & Service,Donut Shop,Grocery Store,Restaurant,Food Court,Food & Drink Shop,Food,Falafel Restaurant
3,Atwater_Village,53872,762,0.9,34,0.34,0.06,0.20,0.01,0.51,0.22,14888,1.8,-118.265808,34.124908,0.0,Mobile Phone Shop,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Food Truck,Breakfast Spot,Sushi Restaurant,Gastropub,Steakhouse,Miscellaneous Shop
4,Baldwin_Hills/Crenshaw,37948,656,0.4,36,0.36,0.10,0.05,0.71,0.17,0.03,30123,3.0,-118.366700,34.019090,0.0,Flower Shop,Clothing Store,Women's Store,Farmers Market,French Restaurant,Fountain,Food Truck,Food Stand,Food Service,Food Court


Finally, let's visualize the resulting clusters

In [37]:
la_merged = la_merged.dropna(subset=['Cluster Labels'])
la_merged = la_merged.astype({'Cluster Labels': int})


In [38]:
la_merged

,Neighborhood,Income,Schools,Diversity,Age,Homes,Vets,Asian,Black,Latino,White,Population,Area,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams_Normandie,29606,691,0.6,26,0.26,0.05,0.05,0.25,0.62,0.06,31068,0.8,-118.300270,34.030970,0,Sushi Restaurant,Grocery Store,Home Service,Pharmacy,Latin American Restaurant,Gas Station,Taco Place,Playground,Food,Park
1,Arleta,65649,719,0.4,29,0.29,0.07,0.11,0.02,0.72,0.13,31068,3.1,-118.430015,34.240603,0,Bakery,Video Store,Historic Site,Convenience Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop
2,Arlington_Heights,31423,687,0.8,31,0.31,0.05,0.13,0.25,0.57,0.05,22106,1.0,-118.320109,34.043611,0,Rental Car Location,Café,Shop & Service,Donut Shop,Grocery Store,Restaurant,Food Court,Food & Drink Shop,Food,Falafel Restaurant
3,Atwater_Village,53872,762,0.9,34,0.34,0.06,0.20,0.01,0.51,0.22,14888,1.8,-118.265808,34.124908,0,Mobile Phone Shop,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Food Truck,Breakfast Spot,Sushi Restaurant,Gastropub,Steakhouse,Miscellaneous Shop
4,Baldwin_Hills/Crenshaw,37948,656,0.4,36,0.36,0.10,0.05,0.71,0.17,0.03,30123,3.0,-118.366700,34.019090,0,Flower Shop,Clothing Store,Women's Store,Farmers Market,French Restaurant,Fountain,Food Truck,Food Stand,Food Service,Food Court
5,Bel-Air,208861,924,0.2,46,0.46,0.13,0.08,0.01,0.05,0.83,7928,6.6,-118.463558,34.096148,0,Dive Bar,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Women's Store,Falafel Restaurant
6,Beverly_Crest,168104,0,0.1,45,0.45,0.10,0.04,0.02,0.03,0.88,10610,7.9,-118.424713,34.112107,0,Food Truck,Women's Store,Falafel Restaurant,French Restaurant,Fountain,Food Stand,Food Service,Food Court,Food & Drink Shop,Food
7,Beverly_Grove,63039,791,0.2,38,0.38,0.05,0.05,0.02,0.06,0.82,21417,1.7,-118.372111,34.076009,0,Clothing Store,Juice Bar,Cosmetics Shop,Gym / Fitness Center,Vegetarian / Vegan Restaurant,Lingerie Store,Café,Seafood Restaurant,Sushi Restaurant,Chinese Restaurant
8,Beverlywood,105253,872,0.2,39,0.39,0.08,0.07,0.04,0.06,0.80,6080,0.8,-118.395011,34.041610,0,Coffee Shop,Food Truck,Park,Restaurant,Boutique,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market
9,Boyle_Heights,33235,689,0.1,25,0.25,0.03,0.02,0.01,0.94,0.02,92785,6.5,-118.205370,34.033700,0,Grocery Store,Juice Bar,Gym,Liquor Store,Taco Place,Theater,Breakfast Spot,Mobile Phone Shop,Seafood Restaurant,Home Service


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['Neighborhood'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

# Cluster 1

In [42]:
la_merged.loc[la_merged['Cluster Labels'] == 0, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Income,Homes,Vets,Asian,Black,Latino,White,Population,Area,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,29606,0.26,0.05,0.05,0.25,0.62,0.06,31068,0.8,-118.300270,34.030970,0,Sushi Restaurant,Grocery Store,Home Service,Pharmacy,Latin American Restaurant,Gas Station,Taco Place,Playground,Food,Park
1,65649,0.29,0.07,0.11,0.02,0.72,0.13,31068,3.1,-118.430015,34.240603,0,Bakery,Video Store,Historic Site,Convenience Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop
2,31423,0.31,0.05,0.13,0.25,0.57,0.05,22106,1.0,-118.320109,34.043611,0,Rental Car Location,Café,Shop & Service,Donut Shop,Grocery Store,Restaurant,Food Court,Food & Drink Shop,Food,Falafel Restaurant
3,53872,0.34,0.06,0.20,0.01,0.51,0.22,14888,1.8,-118.265808,34.124908,0,Mobile Phone Shop,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Food Truck,Breakfast Spot,Sushi Restaurant,Gastropub,Steakhouse,Miscellaneous Shop
4,37948,0.36,0.10,0.05,0.71,0.17,0.03,30123,3.0,-118.366700,34.019090,0,Flower Shop,Clothing Store,Women's Store,Farmers Market,French Restaurant,Fountain,Food Truck,Food Stand,Food Service,Food Court
5,208861,0.46,0.13,0.08,0.01,0.05,0.83,7928,6.6,-118.463558,34.096148,0,Dive Bar,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Women's Store,Falafel Restaurant
6,168104,0.45,0.10,0.04,0.02,0.03,0.88,10610,7.9,-118.424713,34.112107,0,Food Truck,Women's Store,Falafel Restaurant,French Restaurant,Fountain,Food Stand,Food Service,Food Court,Food & Drink Shop,Food
7,63039,0.38,0.05,0.05,0.02,0.06,0.82,21417,1.7,-118.372111,34.076009,0,Clothing Store,Juice Bar,Cosmetics Shop,Gym / Fitness Center,Vegetarian / Vegan Restaurant,Lingerie Store,Café,Seafood Restaurant,Sushi Restaurant,Chinese Restaurant
8,105253,0.39,0.08,0.07,0.04,0.06,0.80,6080,0.8,-118.395011,34.041610,0,Coffee Shop,Food Truck,Park,Restaurant,Boutique,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market
9,33235,0.25,0.03,0.02,0.01,0.94,0.02,92785,6.5,-118.205370,34.033700,0,Grocery Store,Juice Bar,Gym,Liquor Store,Taco Place,Theater,Breakfast Spot,Mobile Phone Shop,Seafood Restaurant,Home Service


# Cluster 2


In [44]:
la_merged.loc[la_merged['Cluster Labels'] == 1, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Income,Homes,Vets,Asian,Black,Latino,White,Population,Area,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,37013,0.28,0.07,0.01,0.48,0.48,0.01,10297,0.6,-118.304507,33.983113,1,Food,Women's Store,Falafel Restaurant,French Restaurant,Fountain,Food Truck,Food Stand,Food Service,Food Court,Food & Drink Shop


# Cluster 3


In [46]:
la_merged.loc[la_merged['Cluster Labels'] == 2, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Income,Homes,Vets,Asian,Black,Latino,White,Population,Area,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,65783,0.28,0.08,0.03,0.04,0.70,0.21,69499,12.4,-118.456816,34.313000,2,Health & Beauty Service,Garden,Business Service,Liquor Store,Women's Store,Food,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop
87,73195,0.38,0.09,0.05,0.04,0.15,0.71,35502,8.8,-118.550118,34.164405,2,Business Service,Women's Store,Farmers Market,French Restaurant,Fountain,Food Truck,Food Stand,Food Service,Food Court,Food & Drink Shop


# Cluster 4

In [48]:
la_merged.loc[la_merged['Cluster Labels'] == 3, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Income,Homes,Vets,Asian,Black,Latino,White,Population,Area,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,83911,0.37,0.11,0.16,0.03,0.21,0.56,50535,15.3,-118.505173,34.291746,3,Construction & Landscaping,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Women's Store,Falafel Restaurant


# Cluster 5

In [50]:
la_merged.loc[la_merged['Cluster Labels'] == 4, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Income,Homes,Vets,Asian,Black,Latino,White,Population,Area,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,31559,0.22,0.02,0.01,0.13,0.85,0.01,40947,2.2,-118.24900,33.99800,4,Clothing Store,Park,Women's Store,Farmers Market,Fountain,Food Truck,Food Stand,Food Service,Food Court,Food & Drink Shop
61,55901,0.51,0.05,0.17,0.03,0.66,0.12,16768,2.4,-118.19027,34.09534,4,Park,Lake,Women's Store,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Food,Falafel Restaurant


In [51]:
la_grouped2 = la_grouped

In [52]:
la_grouped2 = la_grouped2.drop([5, 35])

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted2 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted2['Neighborhood'] = la_grouped2['Neighborhood']

for ind in np.arange(la_grouped2.shape[0]):
    neighborhoods_venues_sorted2.iloc[ind, 1:] = return_most_common_venues(la_grouped2.iloc[ind, :], num_top_venues)


kclusters = 5

la_grouped_clustering2 = la_grouped2.drop('Neighborhood', 1)

# run k-means clustering
kmeans2 = KMeans(n_clusters=kclusters, random_state=0).fit(la_grouped_clustering2)


neighborhoods_venues_sorted2.insert(0, 'Cluster Labels', kmeans2.labels_)

la_merged2 = LA

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
la_merged2 = la_merged2.join(neighborhoods_venues_sorted2.set_index('Neighborhood'), on='Neighborhood')

la_merged2 = la_merged2.dropna(subset=['Cluster Labels'])
la_merged2 = la_merged2.astype({'Cluster Labels': int})

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged2['Latitude'], la_merged2['Longitude'], la_merged2['Neighborhood'], la_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

ValueError: Length of values does not match length of index